### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
import ads

In [2]:
ads.__version__

'2.10.0'

In [3]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "./taln_pii_cs_models_local/taln_pii_cs_model"

data = "Mon nom est Hicham, et pour l'anniversaire de 50ans et je vais payer avec une carte visa 4442223314488."
model = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="first" #"simple"
)
preds = model(data)
preds

[{'entity_group': 'FIRSTNAME',
  'score': 0.44788054,
  'word': 'Hicham,',
  'start': 12,
  'end': 19},
 {'entity_group': 'CREDITCARDNUMBER',
  'score': 0.3592835,
  'word': '4442223314488.',
  'start': 89,
  'end': 103}]

#### Prepare Model Artifact
Instantiate a HuggingFacePipelineModel() object with HuggingFace pipelines. All the pipelines related files are saved under the artifact_dir.

In [4]:
from ads.common.model_metadata import UseCaseType
from ads.model import HuggingFacePipelineModel


import tempfile

#import oci
#signer = oci.auth.signers.get_resource_principals_signer()
ads.set_auth(auth='resource_principal')

# Prepare the model
artifact_dir = "./taln_pii_cs_models"
huggingface_pipeline_model = HuggingFacePipelineModel(model, artifact_dir=artifact_dir)
huggingface_pipeline_model.prepare(
  inference_conda_env="oci://taln_pii_cs_conda_envs@yz2wwgkgt8eh/conda_environments/gpu/PyTorch 2.1 for GPU on Python 3.9/1.0/pytorch21_p39_gpu_v1",
  inference_python_version="3.9",
  training_conda_env="oci://taln_pii_cs_conda_envs@yz2wwgkgt8eh/conda_environments/gpu/PyTorch 2.1 for GPU on Python 3.9/1.0/pytorch21_p39_gpu_v1",
  use_case_type=UseCaseType.OTHER,
  force_overwrite=True,
  )
# You don't need to modify the score.py generated. The model can be loaded by the transformers.pipeline.
# More info here - https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline

algorithm: TokenClassificationPipeline
artifact_dir:
  /home/datascience/taln_pii_cs_models:
  - - sentencepiece.bpe.model
    - added_tokens.json
    - pytorch_model.bin
    - score.py
    - test_json_output.json
    - .model-ignore
    - tokenizer.json
    - model.safetensors
    - runtime.yaml
    - special_tokens_map.json
    - config.json
    - tokenizer_config.json
framework: transformers
model_deployment_id: null
model_id: null

#### Manually correct score.py

1- In the method load_model, add a correction that introduces an aggregation_strategy parameter for pipeline, specifically set to "first"
comment out the original line:
```
        # model = pipeline(task, model = model_dir)
```
Add the new lines below the commented line:
```
        #hicham's fix to add ner aggregation_strategy
        model = pipeline(task, model = model_dir, aggregation_strategy="first")
```
2- In the method serialize_prediction, add a correction that fixes the error "The inference result is not json parsable Object of type float32 is not JSON serializable"
```
    """
    if isinstance(yhat, dict):
        for key, value in yhat.items():
            if isinstance(value, torch.Tensor):
                yhat[key] = value.tolist()
            elif str(type(value)).startswith("<class 'PIL."):
                import PIL
                yhat[key] = np.asarray(value).tolist() if isinstance(value, PIL.Image.Image) else value
    """
    
    #hicham's fix for the error "The inference result is not json parsable Object of type float32 is not JSON serializable" 
    if isinstance(yhat, dict):
        for key, value in yhat.items():
            if isinstance(value, np.float32):
                yhat[key] = float(value)
```


In [5]:
huggingface_pipeline_model.introspect()

['sentencepiece.bpe.model', 'added_tokens.json', 'pytorch_model.bin', 'score.py', 'test_json_output.json', '.model-ignore', 'tokenizer.json', 'model.safetensors', 'runtime.yaml', 'special_tokens_map.json', 'config.json', 'tokenizer_config.json']


,Test key,Test name,Result,Message
0,runtime_env_path,Check that field MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is set,Passed,
1,runtime_env_python,Check that field MODEL_DEPLOYMENT.INFERENCE_PYTHON_VERSION is set to a value of 3.6 or higher,Passed,
2,runtime_path_exist,Check that the file path in MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is correct.,Passed,
3,runtime_version,Check that field MODEL_ARTIFACT_VERSION is set to 3.0,Passed,
4,runtime_yaml,"Check that the file ""runtime.yaml"" exists and is in the top level directory of the artifact directory",Passed,
5,score_load_model,Check that load_model() is defined,Passed,
6,score_predict,Check that predict() is defined,Passed,
7,score_predict_arg,Check that all other arguments in predict() are optional and have default values,Passed,
8,score_predict_data,"Check that the only required argument for predict() is named ""data""",Passed,
9,score_py,"Check that the file ""score.py"" exists and is in the top level directory of the artifact directory",Passed,


##### Call model summary
The .summary_status() method returns a Pandas dataframe that guides you through the entire workflow. It shows which methods are available to call and which ones aren’t. Plus it outlines what each method does. If extra actions are required, it also shows those actions.

In [6]:
huggingface_pipeline_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Available     Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

##### Verify the generated model artifacts 
without deploying the model to model catalogue

In [14]:
huggingface_pipeline_model.verify(data)

Model is successfully loaded.
  warnings.warn(



{'prediction': [{'entity_group': 'FIRSTNAME',
   'score': 0.4478805363178253,
   'word': 'Hicham,',
   'start': 12,
   'end': 19},
  {'entity_group': 'CREDITCARDNUMBER',
   'score': 0.3592835068702698,
   'word': '4442223314488.',
   'start': 89,
   'end': 103}]}

#### Register Model

In [15]:
# Register the model

model_id = huggingface_pipeline_model.save(display_name="TALN PII Case Study Model-pytorch8.0")

Model is successfully loaded.
['sentencepiece.bpe.model', 'added_tokens.json', 'pytorch_model.bin', 'score.py', 'test_json_output.json', '.model-ignore', 'tokenizer.json', 'model.safetensors', 'runtime.yaml', 'special_tokens_map.json', 'config.json', 'tokenizer_config.json']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

#### Deploy and Generate Endpoint

In [16]:
# Deploy and create an endpoint for the huggingface_pipeline_model
huggingface_pipeline_model.deploy(
    display_name="TALN PII Case Study Model-deployment-pytorch8.0", #HuggingFace Pipeline Model
    deployment_log_group_id="ocid1.loggroup.oc1.ca-toronto-1.amaaaaaa3hvgr2qa4t42gdsnhm6ijpsfzhkizrzpp3cbfhvds2fnm2ejczaq",
    deployment_access_log_id="ocid1.log.oc1.ca-toronto-1.amaaaaaa3hvgr2qajbioaw6ru5o4hzt5q767rngzprsojfqemqvsoaa3cj6a",
    deployment_predict_log_id="ocid1.log.oc1.ca-toronto-1.amaaaaaa3hvgr2qajbioaw6ru5o4hzt5q767rngzprsojfqemqvsoaa3cj6a",
    # Shape config details mandatory for flexible shapes:
    # deployment_instance_shape="VM.Standard.E4.Flex",
    # deployment_ocpus=<number>,
    # deployment_memory_in_gbs=<number>,
)
print(f"Endpoint: {huggingface_pipeline_model.model_deployment.url}")
# Output: "Endpoint: https://modeldeployment.{region}.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.xxx.xxxxx"

Model Deployment OCID: ocid1.datasciencemodeldeployment.oc1.ca-toronto-1.amaaaaaa3hvgr2qazlfu5f5vwmt7k7j5bp3bpn7la73tnvqrca36ibtcmhla


Creating model deployment:   0%|          | [00:00<?, ?it/s]

Endpoint: https://modeldeployment.ca-toronto-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.ca-toronto-1.amaaaaaa3hvgr2qazlfu5f5vwmt7k7j5bp3bpn7la73tnvqrca36ibtcmhla


### Run Prediction against Endpoint

In [25]:
# Generate prediction by invoking the deployed endpoint
datab= ["2test hicham ici"]
preds = huggingface_pipeline_model.predict(data)

#print predictions
for pred in preds['prediction']:
    print(pred['word'],pred['entity_group'], pred['score'], pred['start'], pred['end'])

Hicham, FIRSTNAME 0.4478805363178253 12 19
4442223314488. CREDITCARDNUMBER 0.3592835068702698 89 103


In [28]:
data

"Mon nom est Hicham, et pour l'anniversaire de 50ans et je vais payer avec une carte visa 4442223314488."

In [42]:
bdata = "data = b\"\\x80\\x05\\x95k\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x8cgMon nom est Hicham, et pour l'anniversaire de 50ans et je vais payer avec une carte visa 4442223314488.\\x94.\""
bdata

'data = b"\\x80\\x05\\x95k\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x8cgMon nom est Hicham, et pour l\'anniversaire de 50ans et je vais payer avec une carte visa 4442223314488.\\x94."'

In [54]:
# The OCI SDK must be installed for this example to function properly.
# Installation instructions can be found here: https://docs.oracle.com/en-us/iaas/Content/API/SDKDocs/pythonsdk.htm

import requests
import oci
from oci.signer import Signer

config = oci.config.from_file("~/.oci/config") # replace with the location of your oci config file
auth = Signer(
  tenancy=config['tenancy'],
  user=config['user'],
  fingerprint=config['fingerprint'],
  private_key_file_location=config['key_file'],
  pass_phrase=config['pass_phrase'])

endpoint = 'https://modeldeployment.ca-toronto-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.ca-toronto-1.amaaaaaa3hvgr2qazlfu5f5vwmt7k7j5bp3bpn7la73tnvqrca36ibtcmhla/predict'
body = {"inputs":data} # payload goes here
headers = {} # header goes here

requests.post(endpoint, json=body, auth=auth, headers=headers).json()

{'prediction': [{'entity_group': 'FIRSTNAME',
   'score': 0.4478805363178253,
   'word': 'Hicham,',
   'start': 12,
   'end': 19},
  {'entity_group': 'CREDITCARDNUMBER',
   'score': 0.3592835068702698,
   'word': '4442223314488.',
   'start': 89,
   'end': 103}]}

#### To do:
1- None